# DATA LECTURE AND QUALITY ANALYSIS FROM PYTHON

In [1]:
#Packages:
import pyodbc
import pandas
import string
import psycopg2

In [2]:
conn = psycopg2.connect(host="postgres", 
                        port = 5432, 
                        database="minjusticia", 
                        user="team77", 
                        password="mintic2020.")

cur = conn.cursor()
cur.execute("""SELECT id_delito, id_subtitulo_delito, nombre FROM public.delito;""")
query_results = cur.fetchall()
print(query_results)
cur.close()
conn.close()

[(1, 21, 'ABANDONO DEL PUESTO'), (2, 21, 'ABANDONO DEL SERVICIO'), (3, 3, 'ABIGEATO'), (4, 15, 'ABORTO'), (5, 15, 'ABORTO SIN CONSENTIMIENTO'), (6, 3, 'ABUSO DE CONDICIONES DE INFERIORIDAD'), (7, 3, 'ABUSO DE CONFIANZA'), (8, 5, 'ABUSO DE FUNCION PUBLICA'), (9, 11, 'ACCESO ABUSIVO A UN SISTEMA INFORMATICO'), (10, 20, 'ACCESO ABUSIVO A UN SISTEMA INFORMATICO'), (11, 12, 'ACCESO CARNAL ABUSIVO CON MENOR DE CATORCE AÑOS'), (12, 12, 'ACCESO CARNAL O ACTO SEXUAL ABUSIVOS CON INCAPAZ DE RESISTIR'), (13, 12, 'ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR'), (14, 12, 'ACCESO CARNAL VIOLENTO'), (15, 25, 'ACCESO CARNAL VIOLENTO CONTRA MENOR DE EDAD  (NOTA:SOLO PARA JURISDICCION INDIGENA)'), (16, 19, 'ACCESO CARNAL VIOLENTO EN PERSONA PROTEGIDA'), (17, 12, 'ACOSO SEXUAL'), (18, 12, 'ACTO SEXUAL VIOLENTO'), (19, 19, 'ACTOS DE BARBARIE'), (20, 19, 'ACTOS DE TERRORISMO'), (21, 12, 'ACTOS SEXUALES CON MENOR DE CATORCE AÑOS'), (22, 19, 'ACTOS SEXUALES VIOLENTOS EN PERSONA PR